In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import utils
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Input, Dense, Flatten, Concatenate
from keras.callbacks import ModelCheckpoint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
from IPython.display import Image

import os

In [2]:
np.random.seed(10)

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
((x_train, y_train), (x_test, y_test)) = fashion_mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000, 10)
(10000, 28, 28)
(10000, 10)


In [102]:
class Random_Finetune_ResNet50():
    def __init__(self, input_shape):

        self.fitness = 0
        self.loss = 1000
        
        IMG_SHAPE = input_shape + (3,)
        self.base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
        sample_arr = [True, False]
        self.bool_arr = np.random.choice(sample_arr, size=len(self.base_model.layers))
        self.update_trainable()
        # self.base_model.trainable = True
        # for idx, i in enumerate(self.base_model.layers):
        #     i.trainable = self.bool_arr[idx]
    
    def update_trainable(self, bool_arr=None):
        if bool_arr is not None: 
            self.bool_arr = bool_arr
        self.base_model.trainable = True
        for idx, i in enumerate(self.base_model.layers):
            i.trainable = self.bool_arr[idx]
        
    def forward(self, learning_rate=0.001):
        inputs = Input((28, 28, 1))
        resized_x = tf.keras.layers.experimental.preprocessing.Resizing(32, 32)(inputs)
        first_conv_layer = Conv2D(3, 1, padding='same', activation=None)(resized_x)

        x = self.base_model(first_conv_layer, training = False)
        x = Flatten()(x)
        outputs = Dense(10, activation = 'softmax')(x)

        model = tf.keras.Model(inputs, outputs, name="fashion_mnist_resnet50_model")

        # 'categorical_crossentropy'은 y[0]=[0, 0, 0, 0, 0, 0, 0, 0, 1], y[1, 0, 0, 0, 0, 0, 0, 0, 0]과 같이 one-hot-encoding label일 경우에 사용
        model.compile(loss="categorical_crossentropy", 
        optimizer=tf.keras.optimizers.Adam(learning_rate= learning_rate), 
        metrics=['accuracy'])
        
        return model
    
    def train_model(self, model, train_data, train_targets, validation_data=(x_test, y_test), epochs=20, batch_size=256):
    
        early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
        checkpoint_best_path = 'model_checkpoints_best/checkpoint'
        checkpoint_best = ModelCheckpoint(filepath=checkpoint_best_path,
                                        save_weights_only=True,
                                        save_freq='epoch',
                                        monitor='val_accuracy',
                                        save_best_only=True,
                                        verbose=1)
        history = model.fit(train_data, train_targets,
                        validation_data = validation_data,
                        epochs = epochs,
                        batch_size = batch_size,
                        verbose = 1,
                        callbacks=[early])
        return history

In [77]:
# genomes list에 10개의 np.random.choice로 선정된 ResNet Freezing/Trainable Layer 초기 인구 생성
genomes = [Random_Finetune_ResNet50((32,32)) for _ in range(10)]
best_genomes = None

In [78]:
# 생성한 유전자 랜덤 값이 같은지 다른지 0번과 1번 비교
(genomes[0].bool_arr == genomes[1].bool_arr ).all()

False

In [79]:
# fitness는 val_accuracy로 구성
# score는 val_loss로 구성
# genomes에 최종 학습이 끝난 epoch마다의 val_accuracy, val_loss, bool_arr(layer별 얼릴 것인가 학습할 것인가) 정보가 포함되고 정렬됨

n_gen = 0
for i, genome in enumerate(genomes):
    n_gen += 1
    model = genome.forward(0.0001)
    history = genome.train_model(model, x_train, y_train, (x_test, y_test), 10, 256)
    fitness = history.history['val_accuracy']
    loss = history.history['val_loss']
    
    genome.fitness = fitness
    genome.loss = loss

    print('Generation #%s, Genome #%s, Fitness: %s, Score: %s' % (n_gen, i, genome.fitness, genome.loss))
if best_genomes is not None:
    genomes.extend(best_genomes)
genomes.sort(key=lambda x: x.fitness, reverse=True)

print('===== Generaton #%s\tBest Fitness %s =====' % (n_gen, genomes[0].fitness))

Epoch 1/10
235/235 [==============================] - 23s 74ms/step - loss: 1.4942 - accuracy: 0.4351 - val_loss: 0.9598 - val_accuracy: 0.6211
Epoch 2/10
235/235 [==============================] - 16s 69ms/step - loss: 0.6448 - accuracy: 0.7521 - val_loss: 0.6340 - val_accuracy: 0.7577
Epoch 3/10
235/235 [==============================] - 16s 69ms/step - loss: 0.5441 - accuracy: 0.7911 - val_loss: 0.7479 - val_accuracy: 0.7416
Epoch 4/10
235/235 [==============================] - 16s 70ms/step - loss: 0.4759 - accuracy: 0.8198 - val_loss: 0.4581 - val_accuracy: 0.8293
Epoch 5/10
235/235 [==============================] - 16s 70ms/step - loss: 0.4385 - accuracy: 0.8354 - val_loss: 0.4269 - val_accuracy: 0.8407
Epoch 6/10
235/235 [==============================] - 16s 69ms/step - loss: 0.4037 - accuracy: 0.8482 - val_loss: 0.4304 - val_accuracy: 0.8407
Epoch 7/10
235/235 [==============================] - 16s 69ms/step - loss: 0.3851 - accuracy: 0.8561 - val_loss: 0.4373 - val_accuracy:

In [96]:
print(genomes[4].fitness[-1])
print(genomes[4].loss[-1])
print(type(genomes[4]))

0.8640999794006348
0.36328595876693726
<class '__main__.Random_Finetune_ResNet50'>


In [83]:
# genomes에 최종 학습이 끝난 epoch마다의 val_accuracy, val_loss, bool_arr(layer별 얼릴 것인가 학습할 것인가) 정보가 포함되고 정렬됨
# genomes에서 가장 좋은 accuracy와 그에 대응하는 bool_arr를 추출
# best_accuracy = genomes별로 가장 좋은 accuracy만 포함
# best_bool = genomes별 bool_arr
# best_accuracy와 best_bool은 pair임

best_accuracy = np.array([])
# best_bool = np.array([])
best_bool = []

for genome in genomes:
    best_fitness = genome.fitness[-1]
    best_bool_arr = genome.bool_arr
    best_accuracy = np.append(best_accuracy, best_fitness)
    # best_bool = np.append(best_bool, best_bool_arr)
    best_bool.append(best_bool_arr)

idx = best_accuracy.argmax()
best_fine_tune = best_bool[idx]
print(idx)

4


In [84]:
best_accuracy_1 = best_accuracy.copy()
best_bool_1 = best_bool.copy()

In [85]:
test = np.sort(best_accuracy_1)
print(test)
print(best_accuracy_1)

[0.79979998 0.81510001 0.8168     0.83240002 0.83770001 0.84289998
 0.8441     0.84729999 0.86260003 0.86409998]
[0.86260003 0.8441     0.83770001 0.84729999 0.86409998 0.79979998
 0.84289998 0.83240002 0.81510001 0.8168    ]


In [86]:
# best_accuracy와 best_bool은 pair임
# pair를 정렬해서 ratio만큼 상위 accuracy에 해당하는 bool_arr를 추출

pair = zip(best_accuracy_1, best_bool_1)
best_pair = sorted(pair, reverse=True)
ratio = 0.6
flag = round(len(best_pair) * ratio)
print(flag)
top_pair = best_pair[0:flag]

6


In [106]:
print(top_pair)

[(0.8640999794006348, array([ True, False,  True, False, False, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True, False,  True,  True,  True, False,  True,
        True, False, False,  True, False, False,  True, False, False,
        True, False, False,  True, False,  True, False, False,  True,
       False, False, False,  True, False,  True, False, False, False,
       False, False,  True, False, False, False, False, False,  True,
        True, False,  True, False, False, False,  True, False,  True,
       False,  True,  True,  True,  True, False, False,  True,  True,
       False,  True,  True,  True, False, False,  True,  True,  True,
       False,  True,  True, False, False, False, False, False, False,
        True,  True,  True,  True, False,  True,  True, False,  True,
        True,  True,  True,  True, False,  True,  True, False, False

In [105]:
import pickle

# save
with open('test_pair_data.pickle', 'wb') as f:
    pickle.dump(top_pair, f, pickle.HIGHEST_PROTOCOL)

In [107]:
# load
with open('test_pair_data.pickle', 'rb') as f:
    top_pair = pickle.load(f)
print(top_pair)

[(0.8640999794006348, array([ True, False,  True, False, False, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True, False,  True,  True,  True, False,  True,
        True, False, False,  True, False, False,  True, False, False,
        True, False, False,  True, False,  True, False, False,  True,
       False, False, False,  True, False,  True, False, False, False,
       False, False,  True, False, False, False, False, False,  True,
        True, False,  True, False, False, False,  True, False,  True,
       False,  True,  True,  True,  True, False, False,  True,  True,
       False,  True,  True,  True, False, False,  True,  True,  True,
       False,  True,  True, False, False, False, False, False, False,
        True,  True,  True,  True, False,  True,  True, False,  True,
        True,  True,  True,  True, False,  True,  True, False, False

In [87]:
# unzip하여 top_accuracy에 해당하는 top_bool 정보를 기억
top_accuracy, top_bool = zip(*top_pair)

In [95]:
# zip하기 전 값과 unzip한 값을 비교하여 정렬여부 확인
print(top_pair[5][0] is top_accuracy[5])
print(top_pair[5][1] is top_bool[5])
print(type(top_pair))

True
True
<class 'list'>


In [90]:
from copy import deepcopy

best_genomes = deepcopy(top_pair)

In [94]:
print(len(best_genomes[0][1]))

a_genome = np.random.choice(best_genomes[0][1], size=len(best_genomes[0][1]))
print(a_genome is best_genomes[0][1])

cut = np.random.randint(0, len(best_genomes[0][1]))
print(cut)

175
False
61


In [101]:
# crossover
a_genome = deepcopy(best_genomes[0][1])
b_genome = deepcopy(best_genomes[1][1])

new_genome = []
cut = np.random.randint(0, len(best_genomes[0][1]))
new_genome.extend(a_genome[:cut])
new_genome.extend(b_genome[cut:])

print(cut)
print(len(new_genome))
print(new_genome)

34
175
[True, False, True, False, False, False, True, True, True, False, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, False, True, False, True, True, True, False, True, False, False, False, True, False, True, True, False, True, True, True, True, False, True, False, False, True, False, True, False, True, False, False, False, True, True, False, False, True, False, False, False, False, True, True, False, False, False, False, False, False, True, True, True, True, False, True, False, True, True, False, False, False, False, True, True, True, True, False, True, True, False, True, True, False, False, False, True, False, False, False, False, False, True, True, False, True, False, False, True, True, False, False, True, False, True, True, True, False, False, False, False, False, True, False, False, True, False, False, False, True, True, False, False, True, False, True, True, False, False, False, True, True, False, False, False, False

In [135]:
# crossover Class Level
crossover0 = Random_Finetune_ResNet50((32,32))
print(crossover0.bool_arr)

a_genome = deepcopy(best_genomes[0][1])
b_genome = deepcopy(best_genomes[1][1])

new_genome = []
cut = np.random.randint(0, len(best_genomes[0][1]))
new_genome.extend(a_genome[:cut])
new_genome.extend(b_genome[cut:])

print(cut)
print(len(new_genome))
print(new_genome)

print(crossover0.bool_arr is new_genome)

crossover0.set_bool_arr(new_genome)
print(crossover0.bool_arr is new_genome)

[ True  True  True  True False  True  True  True False  True  True False
  True False False False  True False  True  True  True False False  True
 False  True  True  True  True False  True  True  True False False False
 False  True False  True False False False  True  True False False  True
 False  True False  True  True False  True  True False False  True  True
 False  True False  True False  True  True False  True False False False
 False  True  True False False  True False False False  True  True False
 False  True  True False  True  True  True False  True  True  True  True
 False  True False False  True  True  True  True False  True  True  True
  True  True False False False False  True  True False False  True False
 False  True  True False False  True False  True  True False False False
 False  True False False False False False False  True  True False False
 False  True  True False False  True False  True  True  True  True False
  True False  True  True  True False False  True Fa

In [137]:
# Mutation
import random
PROB_MUTATION = 0.04

mutation = deepcopy(crossover0.bool_arr)
print(mutation)
print(crossover0.bool_arr is mutation)
size = np.random.randint(len(crossover0.bool_arr))
real_size = round(size*PROB_MUTATION)
print(size, real_size)
mutation_arr = random.sample(range(0,len(crossover0.bool_arr)),real_size)
mutation_arr_sort = sorted(mutation_arr)
print(mutation_arr_sort)

print(mutation)
for i in mutation_arr_sort:
    boolen = mutation[i]
    if boolen == True:
        mutation[i] = False
    else:
        mutation[i] = True
print(mutation)
print(crossover0.bool_arr is mutation)


[True, False, True, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, False, True, True, False, False, True, False, True, False, True, True, False, True, False, False, False, True, False, True, True, False, True, True, True, True, False, True, False, False, True, False, True, False, True, False, False, False, True, True, False, False, True, False, False, False, False, True, True, False, False, False, False, False, False, True, True, True, True, False, True, False, True, True, False, False, False, False, True, True, True, True, False, True, True, False, True, True, False, False, False, True, False, False, False, False, False, True, True, False, True, False, False, True, True, False, False, True, False, True, True, True, False, False, False, False, False, True, False, False, True, False, False, False, True, True, False, False, True, False, True, True, False, False, False, True, True, False, False, False,

In [136]:
# Mutation
import random
PROB_MUTATION = 0.04

mutation1 = deepcopy(crossover0.bool_arr)
print(crossover0.bool_arr is mutation1)
print(mutation)
print(crossover0.bool_arr)

if np.random.uniform(0,1) < PROB_MUTATION:

    size = np.random.randint(len(mutation1))
    real_size = round(size*PROB_MUTATION)

    mutation_arr = random.sample(range(0,len(crossover0.bool_arr)),real_size)
    mutation_arr_sort = sorted(mutation_arr)

    for i in mutation_arr_sort:
        boolen = mutation1[i]
        if boolen == True:
            mutation1[i] = False
        else:
            mutation1[i] = True

print(crossover0.bool_arr is mutation1)


False
[True, False, True, False, False, False, True, True, True, False, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, False, True, False, False, True, True, False, True, True, False, False, True, False, False, True, False, False, True, False, False, True, False, True, False, False, True, False, False, False, True, False, True, False, False, False, False, False, True, False, False, False, False, False, True, True, False, True, False, False, False, True, False, True, False, True, True, True, True, False, False, True, True, False, True, True, True, False, False, True, False, True, False, True, False, False, False, False, False, False, False, True, True, True, True, False, True, True, False, True, True, True, True, False, False, True, True, False, False, False, True, False, False, False, True, False, True, False, True, False, True, True, False, True, False, False, True, True, True, True, True, True, False, False, False, False, 

In [ ]:
# crossover
N_CHILDREN =5
for i in range(N_CHILDREN):
    pass